In [27]:
import numpy as np
import random

# تابع ارزیابی


In [28]:
def eval_(chromo, function = "ackley", mutation = "onestep"):
    
    if mutation == "onestep":
        chromo = chromo[0:len(chromo)-1]
    elif mutation == "nstep":
        chromo = chromo[0:int(len(chromo)/2)]
    nx = len(chromo)
    
    
    assert function == "ackley" or function == "beale" or function == "SCHAFFER"  or function == "holder", "The Evaluation Function is incorrect"
    
    if(function == "ackley"):
        fitness = -20 * np.exp(-0.2 * np.sqrt((1/nx) * np.sum(chromo * chromo))) - np.exp((1/nx) * np.sum(np.cos(2 * np.pi * chromo))) + 20 + np.exp(1)
    
    elif(function == "beale"):
        assert len(chromo) == 2, "Beale is a 2 dimensional function https://www.sfu.ca/~ssurjano/beale.html"
        x = chromo[0]
        y = chromo[1]
        fitness = np.power((1.5 - x + x*y),2) + np.power((2.25 - x + x*y*y),2) + np.power((2.625 - x + x*y*y*y),2)
            
    elif(function == "SCHAFFER"):
        assert len(chromo) == 2, "SCHAFFER https://www.sfu.ca/~ssurjano/schaffer2.html"
        x1 = chromo[0]
        x2 = chromo[1]
        fitness = 0.5 + (np.power(np.sin(np.power(x1,2)-np.power(x2,2)),2) - 0.5)/(np.power(1+0.001*np.power(x1,2)+np.power(x2,2),2))
                                                                                   
    elif(function == "holder"):
        assert len(chromo) == 2, "Holder Table is a 2 dimensional function https://www.sfu.ca/~ssurjano/holder.html"
        x1 = chromo[0]
        x2 = chromo[1]
        fitness = -1 * np.abs(np.sin(x1) * np.cos(x2) * np.exp(np.abs(1 - (np.sqrt(x1*x1 + x2*x2))/np.pi)))
    
    return fitness

# ساخت جمعیت اولیه

متغیر های up ,low
محدوده ی مورد نظر که در صورت سوال گفته را مشخص می کند 

In [29]:

def initialize_pop(num_parents = 10, nx = 4, up = 30, low = -30, sigma_up = 2, mutation = "onestep"):
    
    assert mutation == "onestep" or mutation == "nstep", "Mutation type is spelt wrong"
    
    if mutation == "onestep":
        pop = np.empty([num_parents, nx+1])
        for i in range(0,num_parents):
            chromo = []
            for j in range(0, nx):
                chromo.append(random.uniform(low,up))
            chromo.append(random.uniform(0, sigma_up))
            chromo = np.array(chromo)
            pop[i] = chromo
            
    elif mutation == "nstep":
        pop = np.empty([num_parents, nx+nx])
        for i in range(0, num_parents):
            chromo = []
            for j in range(0, nx):
                chromo.append(random.uniform(low,up))
            for j in range(0, nx):
                chromo.append(random.uniform(0, sigma_up))
            chromo = np.array(chromo)
            pop[i] = chromo
            
    return pop  

# روش های بازترکیبی
 چهار روش در این بخش پیاده سازی شده

In [30]:

def crossover_parents(parents, method = "local_uniform"):
    
    num_parents = parents.shape[0]
    chromo_len = parents.shape[1]
    children = np.empty([num_parents*7, chromo_len])
    
    assert method == "local_uniform" or method == "global_uniform" or method == "local_average" or method == "global_average", "The method does not exist"
    
    if method == "local_uniform":
        for i in range(0, num_parents*7):
            idx1 = random.randint(0, num_parents-1)
            idx2 = random.randint(0, num_parents-1)
            mask = [random.randint(0,1) for i in range(0, chromo_len)]
            child_chromo = [parents[idx1][j] if mask[j]==0 else parents[idx2][j] for j in range(0, chromo_len)]
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
            
    elif method == "global_uniform":
        for i in range(0, num_parents*7):
            mask = [random.randint(0,num_parents-1) for k in range(0, chromo_len)]
            child_chromo = [parents[mask[k]][k] for k in range(0, chromo_len)]
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
            
    elif method == "local_average":
        for i in range(0, num_parents*7):
            idx1 = random.randint(0, num_parents-1)
            idx2 = random.randint(0, num_parents-1)
            child_chromo = [(parents[idx1][j]+parents[idx2][j])/2 for j in range(0, chromo_len)]
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
            
    elif method == "global_average":
        for i in range(0, num_parents*7):
            child_chromo = []
            for j in range(0, chromo_len):
                idx1 = random.randint(0, num_parents-1)
                idx2 = random.randint(0, num_parents-1)
                child_chromo.append((parents[idx1][j]+parents[idx2][j])/2)
            child_chromo = np.array(child_chromo)
            children[i] = child_chromo
    
    return children

# روش جهش خود تطبیقی

In [31]:

def mutation_self_adaptive(children, up_bound, low_bound, sigma_up = 2, method = "onestep"):
    
    assert method == "onestep" or method == "nstep", "Mutation type is spelt wrong"
    
    if method == "onestep":
        
        num_pop = children.shape[0]
        nx = children.shape[1] - 1
        
        for i in range(0, num_pop):
            
            sigma = children[i][nx]
            tao = 2/ np.sqrt(nx) 
            sigma_prim = sigma * np.exp(tao * np.random.normal(0, 1))
            if sigma_prim < 0.01:
                sigma_prim = 0.01
            
            children[i][nx] = sigma_prim
            
            for j in range(0, nx):
                children[i][j] = children[i][j] + sigma_prim * np.random.normal(0,1)
                if children[i][j] > up_bound:
                    children[i][j] = up_bound
                if children[i][j] < low_bound:
                    children[i][j] = low_bound
    
    elif method == "nstep":
        
        num_pop = children.shape[0]
        nx = int(children.shape[1]/2)
        
        tao_prim = 1/np.sqrt(2 * nx)
        tao = 1/np.sqrt(2 * np.sqrt(nx))
        
        for i in range(0, num_pop):
            
            normal_noise = tao_prim * np.random.normal(0,1)
            
            for j in range(0, nx):
                
                xi = children[i, j]
                sigmai = children[i, nx+j]
                
                sigmai_prim = sigmai * np.exp(normal_noise + tao * np.random.normal(0,1))
                
                if sigmai_prim < 0.01:
                    sigmai_prim = 0.01
                
                if sigmai_prim > sigma_up:
                    sigmai_prim = sigma_up
                
                xi_prim = xi + sigmai_prim * np.random.normal(0,1)
                
                if xi_prim > up_bound:
                    xi_prim = up_bound
                elif xi_prim < low_bound:
                    xi_prim = low_bound

                children[i, j] = xi_prim
                children[i, nx+j] = sigmai_prim
                
    return children

# انتخاب بازماندگان

In [32]:

def select_survivors(children, method = "ackley", mutation_method = "onestep"):
    
    num_children = children.shape[0]
    num_pop = int(num_children / 7)
    pop = []
    fitnesses = []
    
    for i in range(0, num_children):
        fitnesses.append(eval_(children[i], method, mutation_method))
    
    indices = np.argsort(fitnesses)

    for i in range(0, num_pop):
        pop.append(children[indices[i]])
    pop = np.array(pop)
        
    return pop

# اجرای قسمت جهش ناهمبسته ی چند گامی 

In [24]:
max_iter = 500
upper_bound = 4.5
lower_bound = -4.5
num_parents = 10
nx = 2
sigma_up = 2
function = "beale"

best_fitness = 1000000
best_chromo = np.zeros((1, nx), dtype = "float32")

parents = initialize_pop(num_parents, nx, upper_bound, lower_bound, sigma_up, "onestep")

for i in range(0, max_iter):
    children = crossover_parents(parents, "global_uniform")
    children = mutation_self_adaptive(children, upper_bound, lower_bound, "onestep")
    parents = select_survivors(children, function)
    
    if eval_(parents[0], function) < best_fitness:
        best_chromo = parents[0]
        best_fitness = eval_(best_chromo, function)
    
    if i == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
    elif i % 100 == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
    
    elif i == max_iter-1:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
        print("THE BEST CHROMOSOME:")
        print(str(best_chromo) + " fitness: " + str(best_fitness))

Iteration 1:
[ 2.49244255 -0.08588148  0.04309551] fitness: 1.5229934345676148
[-3.48942325  1.3032202   0.78992846] fitness: 2.818981622211287
[ 2.57973787 -0.28768802  0.25430341] fitness: 3.3330807355616527
[ 0.40521239 -0.8864671   0.64069584] fitness: 8.974529249040758
[ 0.30471746 -0.99327524  0.49491889] fitness: 9.928051277664261
[ 0.00918773 -4.5         0.85889598] fitness: 11.15397129474932
[4.5        0.06028753 2.89920023] fitness: 15.946932427721213
[-0.25016512 -1.49439391  0.02544443] fitness: 22.04522123350617
[-3.50988666  0.92118808  0.04935035] fitness: 22.39288121739754
[-0.53145842 -0.90700135  0.04647627] fitness: 24.43700292276494
Iteration 101:
[3.00880553 0.50244336 0.01      ] fitness: 1.3965003468908597e-05
[2.9931151  0.49916262 0.01      ] fitness: 2.5151513977948327e-05
[3.02750304 0.50700539 0.01      ] fitness: 0.00011910306430460366
[2.97675371 0.49564962 0.01      ] fitness: 0.00013713391641256948
[3.02952267 0.50773144 0.01      ] fitness: 0.00014103

# اجرای قسمت جهش ناهمبسته ی چند گامی 

In [34]:
max_iter = 500
upper_bound = 30
lower_bound = -30
num_parents = 20
nx = 4
sigma_up = 2
function = "ackley"
mutation_method = "nstep"

best_fitness = 1000000
best_chromo = np.zeros((1, nx), dtype = "float32")

parents = initialize_pop(num_parents, nx, upper_bound, lower_bound, sigma_up, "nstep")

for i in range(0, max_iter):
    children = crossover_parents(parents, "global_uniform")
    children = mutation_self_adaptive(children, upper_bound, lower_bound, sigma_up, "nstep")
    parents = select_survivors(children, function, mutation_method)
    
    if eval_(parents[0], function, mutation_method) < best_fitness:
        best_chromo = parents[0]
        best_fitness = eval_(best_chromo, function, mutation_method)
    
    if i == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function, mutation_method)))
            
    elif i % 100 == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function, mutation_method)))
    
    elif i == max_iter-1:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function, mutation_method)))
            
        print(" THE BEST CHROMOSOME:")
        print(str(best_chromo) + " fitness: " + str(best_fitness))

Iteration 1:
[ 1.3145999  -2.29183872  1.60019376 -2.21081034  0.80643202  2.
  2.          2.        ] fitness: 8.301811620109255
[-9.55928729 -3.7613099  -4.18781007  1.14551868  2.          1.81425202
  0.42226482  0.08538922] fitness: 15.12835013792091
[-8.41288801  5.74453241  0.1091126   7.81650684  0.35881911  0.33625373
  0.80391378  0.34472957] fitness: 16.104547586696487
[ -7.93675673   3.04924446  -4.86415405 -13.03982748   0.38955639
   1.98600416   0.13154553   0.31969299] fitness: 16.403926093393675
[-4.22670188 11.72628678  4.32139093 -0.55459151  0.37021255  0.07260422
  0.98699023  0.41766812] fitness: 16.669282754968926
[ 4.13140157 -0.25635418 -3.8861793  16.79231855  1.992804    0.86337383
  0.49338248  0.37943673] fitness: 17.807700879518862
[  3.34295189   9.93037973  -7.32102412 -10.76175838   0.41441803
   0.50427005   1.55636507   2.        ] fitness: 17.95767212370413
[-10.57833122  -7.10746286  -5.60416194  10.00850834   2.
   1.41301624   1.26109219   1.5655

Iteration 401:
[-0.00266415  0.00381162 -0.00352687  0.00220605  0.08111208  0.01
  0.03812086  0.01253089] fitness: 0.012997322707886116
[-0.00101668  0.00117927  0.00509755 -0.00636995  0.0265291   0.01089806
  0.01        0.01      ] fitness: 0.017529742484875488
[-0.00071292  0.00406578 -0.00660736 -0.00541904  0.01        0.01
  0.01241809  0.01365112] fitness: 0.020178894486058585
[-0.00700818  0.00856377  0.00099209 -0.00187696  0.01        0.01911794
  0.0135335   0.01      ] fitness: 0.02422480866180665
[ 0.00537193 -0.00394445  0.00289267 -0.00885098  0.01        0.01
  0.01        0.01      ] fitness: 0.02464677445998875
[-0.00887635 -0.00303671 -0.00571713  0.00478892  0.03358416  0.01
  0.01        0.01596229] fitness: 0.025880497719921625
[-0.01010189 -0.00520878  0.00681602  0.00532432  0.01        0.01
  0.01284361  0.01      ] fitness: 0.03127866368501886
[-0.00848194 -0.00105182 -0.0122222   0.00496746  0.01        0.01155387
  0.01        0.04034205] fitness: 0.03472

In [26]:
max_iter = 500
upper_bound = 100
lower_bound = -100
num_parents = 10
nx = 2
sigma_up = 2
function = "SCHAFFER"

best_fitness = 1000000
best_chromo = np.zeros((1, nx), dtype = "float32")

parents = initialize_pop(num_parents, nx, upper_bound, lower_bound, sigma_up, "onestep")

for i in range(0, max_iter):
    children = crossover_parents(parents, "global_uniform")
    children = mutation_self_adaptive(children, upper_bound, lower_bound, "onestep")
    parents = select_survivors(children, function)
    
    if eval_(parents[0], function) < best_fitness:
        best_chromo = parents[0]
        best_fitness = eval_(best_chromo, function)
    
    if i == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
    elif i % 100 == 0:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
    
    elif i == max_iter-1:
        print("Iteration " + str(i+1) + ":")
        for j in range(0, parents.shape[0]):
            print(str(parents[j]) + " " + "fitness: " + str(eval_(parents[j], function)))
            
        print("THE BEST CHROMOSOME:")
        print(str(best_chromo) + " fitness: " + str(best_fitness))

Iteration 1:
[-89.40940634   6.02096953   4.26950044] fitness: 0.499764702012641
[-88.02304757   9.09245967   0.18093479] fitness: 0.49994141585250523
[91.96343832  9.07601565  0.11284463] fitness: 0.49997698618487113
[-13.56226092   9.43831697   0.21190706] fitness: 0.4999783729183844
[-88.07202304 -12.41463072   0.11911995] fitness: 0.49998149984177276
[ 69.60588519 -12.32483376   0.0900778 ] fitness: 0.4999878814757127
[ 94.77637098 -14.83322231   3.13941696] fitness: 0.4999971835649594
[-14.2364188  -24.0832896    0.91713732] fitness: 0.4999988455412105
[ 69.44779173 -25.7545863    0.07683759] fitness: 0.49999899773096546
[-20.12481658 -26.39505453   0.49018494] fitness: 0.49999940732267373
Iteration 101:
[5.52880819e+01 5.53432679e-02 1.00000000e-02] fitness: 0.4696643839064864
[ 5.53450606e+01 -4.32790057e-03  6.54652369e-02] fitness: 0.46974469067019997
[ 5.53164393e+01 -1.52788608e-01  1.00000000e-02] fitness: 0.4700522605461412
[ 5.55151796e+01 -1.25329524e-01  1.00000000e-02]